In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)
%matplotlib inline

In [2]:
train=pd.read_csv("train/train_data.csv")
label=pd.read_csv("train/train_label.csv")
test=pd.read_csv("test/test_data.csv")

In [19]:
#from sklearn.decomposition import PCA
#pca = PCA()
#pca.fit(train.iloc[:,1:])
# 分析結果を元にデータセットを主成分に変換する
#transformed = pca.fit_transform(train.iloc[:,1:])

In [50]:
#from sklearn.feature_selection import SelectKBest, f_regression,chi2
#selector = SelectKBest(score_func=chi2, k=100) 
#selector.fit(train.iloc[:,1:], label2)
#X_new = SelectKBest(chi2, k=5).fit_transform(train.iloc[:,1:], label2)

In [83]:
def makefeature(df):
    tmp=df.iloc[:,1:].copy()
    df=pd.DataFrame()
    df["max"]=tmp.max(axis=1)
    df["mean"]=tmp.mean(axis=1)
    df["median"]=tmp.median(axis=1)
    df["min"]=tmp.min(axis=1)
    df["std"]=tmp.std(axis=1)
    return df

In [84]:
tmp2=makefeature(train)
train3=pd.concat([train,pd.DataFrame(tmp2)],axis=1)
tmp4=makefeature(test)
test3=pd.concat([test,pd.DataFrame(tmp4)],axis=1)

In [99]:
train3.shape

(5124, 206)

In [98]:
test3.shape

(5125, 206)

In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(label["class"].unique())
label2=le.transform(label["class"]) 

In [18]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedShuffleSplit
import copy

params={}

params={
    'objective': 'multiclass',
    # クラス数は 3
    'num_class': 16,
    'random_state' : 1,
    #"objective": "rmse",
    #"eval_metric": "rmse",
    #"objective": "binary",
    "metric": "multi_error",
    #'num_leaves': 50,
    }

df_train=train.drop("id",axis=1).copy()
df_label=label2

p_test=0
p_eval=0
kfold = 5
score=0
#sss = StratifiedShuffleSplit(n_splits=kfold, test_size=0.1, random_state=9817)
sss = StratifiedShuffleSplit(n_splits=kfold, random_state=9817)
for i, (train_index, test_index) in enumerate(sss.split(df_train, df_label)):
    print('[Fold %d/%d]' % (i + 1, kfold))
    
    X_train, X_valid = df_train.iloc[train_index,:], df_train.iloc[test_index,:]
    y_train, y_valid = df_label[train_index], df_label[test_index]
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid)
    
    watchlist = [(dtrain, 'train'),(dvalid, 'eval')]
    #bst = lgb.train(params, dtrain, num_boost_round=1000,valid_sets=[dtrain, dvalid],early_stopping_rounds=50,verbose_eval=10)
    bst = lgb.train(params, dtrain, num_boost_round=1000,valid_sets=[dvalid],early_stopping_rounds=50,verbose_eval=50)
    p_test+=bst.predict(test3.drop("id",axis=1))
    #d_test = pred_df
    #p_test+=bst.predict(d_test)
    score+=bst.best_score["valid_0"]['multi_error']
    

[Fold 1/5]
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's multi_error: 0.173489
[100]	valid_0's multi_error: 0.155945
[150]	valid_0's multi_error: 0.140351
[200]	valid_0's multi_error: 0.136452
[250]	valid_0's multi_error: 0.128655
[300]	valid_0's multi_error: 0.126706
Early stopping, best iteration is:
[265]	valid_0's multi_error: 0.124756
[Fold 2/5]
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's multi_error: 0.132554
[100]	valid_0's multi_error: 0.107212
[150]	valid_0's multi_error: 0.0974659
[200]	valid_0's multi_error: 0.0916179
[250]	valid_0's multi_error: 0.08577
[300]	valid_0's multi_error: 0.0818713
[350]	valid_0's multi_error: 0.079922
[400]	valid_0's multi_error: 0.0779727
Early stopping, best iteration is:
[358]	valid_0's multi_error: 0.0760234
[Fold 3/5]
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's multi_error: 0.138402
[100]	valid_0's multi_error: 0.116959
[150]	valid_0's multi_erro

In [121]:
p_test=p_test/5
preds2=np.argmax(p_test,axis=1)

In [122]:
#yを文字に戻す
result=le.inverse_transform(preds2)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [124]:
submit=pd.DataFrame()
submit["id"]=test["id"]
submit["class"]=result

In [125]:
submit.to_csv("spe6.csv",index=False)